<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---working-folder-paths" data-toc-modified-id="Setup---working-folder-paths-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - working folder paths</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li><li><span><a href="#Setup---Initialize-LoggingHelper" data-toc-modified-id="Setup---Initialize-LoggingHelper-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Setup - Initialize LoggingHelper</a></span></li></ul></li><li><span><a href="#Find-articles-to-be-loaded" data-toc-modified-id="Find-articles-to-be-loaded-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find articles to be loaded</a></span><ul class="toc-item"><li><span><a href="#Uncompress-files" data-toc-modified-id="Uncompress-files-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Uncompress files</a></span></li><li><span><a href="#Work-with-uncompressed-files" data-toc-modified-id="Work-with-uncompressed-files-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Work with uncompressed files</a></span></li><li><span><a href="#load-a-file-into-memory." data-toc-modified-id="load-a-file-into-memory.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>load a file into memory.</a></span></li><li><span><a href="#build-list-of-all-ObjectTypes" data-toc-modified-id="build-list-of-all-ObjectTypes-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>build list of all ObjectTypes</a></span><ul class="toc-item"><li><span><a href="#Build-sorted-list-of-object-types" data-toc-modified-id="Build-sorted-list-of-object-types-3.4.1"><span class="toc-item-num">3.4.1&nbsp;&nbsp;</span>Build sorted list of object types</a></span></li></ul></li></ul></li><li><span><a href="#TODO" data-toc-modified-id="TODO-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TODO</a></span></li></ul></div>

# Introduction

- Back to [Table of Contents](#Table-of-Contents)

This is a notebook that expands on the OpenCalais code in the file `article_coding.py`, also in this folder.  It includes more sections on selecting publications you want to submit to OpenCalais as an example.  It is intended to be copied and re-used.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [2]:
# python packages
import datetime
import glob
import logging
import lxml
import os
import six
import xml
import xmltodict
import zipfile

## Setup - working folder paths

- Back to [Table of Contents](#Table-of-Contents)

What data are we looking at?

In [3]:
# paper identifier
paper_identifier = "Newsday"
archive_identifier = None

# source
source_paper_folder = "/mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data"
source_paper_path = "{}/{}".format( source_paper_folder, paper_identifier )
source_archive_file = "{}.zip".format( archive_identifier )
source_archive_path = "{}/{}".format( source_paper_path, source_archive_file )

# uncompressed
uncompressed_paper_folder = "/mnt/hgfs/projects/phd/proquest_hnp/uncompressed"
uncompressed_paper_path = "{}/{}".format( uncompressed_paper_folder, paper_identifier )

# make sure an identifier is set before you make a path here.
if ( ( archive_identifier is not None ) and ( archive_identifier != "" ) ):
    
    # identifier is set.
    uncompressed_archive_path = "{}/{}".format( uncompressed_paper_path, archive_identifier )

#-- END check to see if archive_identifier present. --#

In [4]:
%pwd

'/home/jonathanmorgan/work/django/research/work/phd_work/data/article_loading/proquest_hnp/Newsday'

In [5]:
# current working folder
current_working_folder = "/home/jonathanmorgan/work/django/research/work/phd_work/data/article_loading/proquest_hnp/{}".format( paper_identifier )
current_datetime = datetime.datetime.now()
current_date_string = current_datetime.strftime( "%Y-%m-%d" )

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [6]:
logging_file_name = "{}/research-data_load-{}-{}.log.txt".format( current_working_folder, paper_identifier, current_date_string )
logging.basicConfig(
    level = logging.DEBUG,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = logging_file_name,
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)

## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [7]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [8]:
%run $django_init_path

django initialized at 2019-08-12 19:03:30.826186


In [9]:
# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.models import Article
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase


## Setup - Initialize LoggingHelper

- Back to [Table of Contents](#Table-of-Contents)

Create a LoggingHelper instance to use to log debug and also print at the same time.

Preconditions: Must be run after Django is initialized, since `python_utilities` is in the django path.

In [10]:
# python_utilities
from python_utilities.logging.logging_helper import LoggingHelper

# init
my_logging_helper = LoggingHelper()
my_logging_helper.set_logger_name( "proquest_hnp-article-loading-Newsday" )
log_message = None

# Find articles to be loaded

- Back to [Table of Contents](#Table-of-Contents)

Specify which folder of XML files should be loaded into system, then process all files within the folder.

The compressed archives from proquest_hnp just contain publication XML files, no containing folder.

To process:

- **uncompresed paper folder ( `<paper_folder>` )** - make a folder in `/mnt/hgfs/projects/phd/proquest_hnp/uncompressed` for the paper whose data you are working with, named the same as the paper's folder in `/mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data`.

    - for example, for the Boston Globe, name it "`BostonGlobe`".

- **uncompressed archive folder ( `<archive_folder>` )** - inside a given paper's folder in uncompressed, for each archive file, create a folder named the same as the archive file, but with no ".zip" at the end.

    - For example, for the file "`BG_20171002210239_00001.zip`", make a folder named "`BG_20171002210239_00001`".
    - path should be "`<paper_folder>/<archive_name_no_zip>`.

- unzip the archive into this folder:

        unzip <path_to_zip> -d <archive_folder>



## Uncompress files

- Back to [Table of Contents](#Table-of-Contents)

See if the uncompressed paper folder exists.  If not, set flag and create it.

In [9]:
# declare variables
did_uncomp_paper_folder_exist = False

# check if uncompressed paper folder exists.
if not os.path.exists( uncompressed_paper_path ):
    
    # no.  Make it.
    os.makedirs( uncompressed_paper_path )
    did_uncomp_paper_folder_exist = False
    log_message = "CREATED - Uncompressed paper folder {}".format( uncompressed_paper_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    
else:
    
    # yes.  Set flag.
    did_uncomp_paper_folder_exist = True
    log_message = "EXISTS - Uncompressed paper folder {}".format( uncompressed_paper_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    
#-- END check to see if paper folder exists.

CREATED - Uncompressed paper folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday


For each *.zip file in the paper's source folder:

- parse file name from path returned by glob.
- parse the part before ".zip" from the file name.  This is referred to subsequently as the "archive identifier".
- check if folder named the same as the "archive identifier" is present.

    - If no:
    
        - create it.
        - then, uncompress the archive into it.
        
    - If yes:
    
        - output a message.  Don't want to uncompress if it was already uncompressed once.

In [10]:
# declare variables - papers
did_uncomp_paper_folder_exist = None

# declare variables archive (.zip) files.
zip_file_list = None
zip_file_path = None
zip_file_path_parts_list = None
zip_file_name = None
zip_file_name_parts_list = None
archive_identifier = None
uc_archive_folder_path = None
zip_file = None

# declare variables - auditing (uc = uncompressed)
archive_file_counter = None
did_uc_archive_folder_exist = None
uc_folder_exists_counter = None
start_dt = None
end_dt = None
time_delta = None

# use glob to get list of zip files in paper source folder.
zip_file_list = glob.glob( "{}/*.zip".format( source_paper_path ) )
zip_file_count = len( zip_file_list )

log_message = "==> zip file count: {}".format( zip_file_count )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )

# loop over zip files.
archive_file_counter = 0
did_uc_archive_folder_exist = False
uc_folder_exists_counter = 0
for zip_file_path in zip_file_list:
    
    # increment counter
    archive_file_counter += 1
    
    log_message = "----> processing file {} of {}".format( archive_file_counter, zip_file_count )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    
    # get file name
    
    # split path into parts on path separator.
    zip_file_path_parts_list = zip_file_path.split( "/" )

    # file name is the last thing in the list.
    zip_file_name = zip_file_path_parts_list[ -1 ]

    # archive_identifier is name with ".zip" removed from end.
    zip_file_name_parts_list = zip_file_name.split( ".zip" )
    archive_identifier = zip_file_name_parts_list[ 0 ]
    
    # for now, log and print the things we've just created.
    log_message = "==> path: {}".format( zip_file_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    log_message = "==> file: {}".format( zip_file_name )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    log_message = "==> ID: {}".format( archive_identifier )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # check if uncompressed archive folder exists.
    uc_archive_folder_path = "{}/{}".format( uncompressed_paper_path, archive_identifier )

    log_message = "==> TO: {}".format( uc_archive_folder_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # check if the uncompressed archive folder exists.
    did_uc_archive_folder_exist = os.path.exists( uc_archive_folder_path )
    if did_uc_archive_folder_exist == False:

        # no.  Make it.
        os.makedirs( uc_archive_folder_path )
        log_message = "CREATED - Uncompressed archive folder {}".format( uc_archive_folder_path )
        my_logging_helper.output_debug_message( log_message, do_print_IN = True )
        
        # and uncompress archive to it.
        with zipfile.ZipFile( zip_file_path, 'r' ) as zip_file:

            # starting extract.
            start_dt = datetime.datetime.now()
            log_message = "==> extract started at {}".format( start_dt )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
            # unzip to uncompressed archive folder path.
            zip_file.extractall( uc_archive_folder_path )
            
            log_message = "EXTRACTED - {}".format( zip_file_path )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )

            log_message = "TO uncompressed archive folder - {}".format( uc_archive_folder_path )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
            # complete
            end_dt = datetime.datetime.now()
            
            log_message = "==> extract completed at {}".format( end_dt )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
            log_message = "==> time elapsed: {}".format( end_dt - start_dt )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
        #-- END with ZipFile --#

    else:

        # yes.  Set flag.
        uc_folder_exists_counter += 1
        log_message = "EXISTS, so moving on - Uncompressed archive folder {}".format( uncompressed_archive_path )
        my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    #-- END check to see if archive folder exists. --#

    log_message = "------------------------------"
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

#-- END loop over zip files. --#

==> zip file count: 194
----> processing file 1 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006220113_00001.zip
==> file: Newsday_20171006220113_00001.zip
==> ID: Newsday_20171006220113_00001
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220113_00001
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220113_00001
==> extract started at 2019-08-07 02:52:59.761334
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006220113_00001.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220113_00001
==> extract completed at 2019-08-07 02:53:58.689399
==> time elapsed: 0:00:58.928065
------------------------------
----> processing file 2 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006220215_00002.zip
==> file: News

==> extract started at 2019-08-07 03:04:21.367638
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006220435_00011.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220435_00011
==> extract completed at 2019-08-07 03:05:21.752937
==> time elapsed: 0:01:00.385299
------------------------------
----> processing file 12 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006220437_00012.zip
==> file: Newsday_20171006220437_00012.zip
==> ID: Newsday_20171006220437_00012
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220437_00012
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220437_00012
==> extract started at 2019-08-07 03:05:22.322882
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006220437_00012.zip
TO uncompressed 

==> extract started at 2019-08-07 03:14:16.366968
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006221954_00007.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006221954_00007
==> extract completed at 2019-08-07 03:15:39.311469
==> time elapsed: 0:01:22.944501
------------------------------
----> processing file 22 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222000_00010.zip
==> file: Newsday_20171006222000_00010.zip
==> ID: Newsday_20171006222000_00010
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222000_00010
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222000_00010
==> extract started at 2019-08-07 03:15:39.887441
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222000_00010.zip
TO uncompressed 

==> extract started at 2019-08-07 03:28:17.987893
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222216_00017.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222216_00017
==> extract completed at 2019-08-07 03:29:37.089948
==> time elapsed: 0:01:19.102055
------------------------------
----> processing file 32 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222319_00018.zip
==> file: Newsday_20171006222319_00018.zip
==> ID: Newsday_20171006222319_00018
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222319_00018
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222319_00018
==> extract started at 2019-08-07 03:29:37.563969
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222319_00018.zip
TO uncompressed 

==> extract started at 2019-08-07 03:42:00.093454
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222537_00027.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222537_00027
==> extract completed at 2019-08-07 03:43:15.680188
==> time elapsed: 0:01:15.586734
------------------------------
----> processing file 42 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222639_00028.zip
==> file: Newsday_20171006222639_00028.zip
==> ID: Newsday_20171006222639_00028
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222639_00028
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222639_00028
==> extract started at 2019-08-07 03:43:16.169863
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006222639_00028.zip
TO uncompressed 

...

==> extract started at 2019-08-07 06:06:01.656043
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006230522_00008.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230522_00008
==> extract completed at 2019-08-07 06:07:25.564980
==> time elapsed: 0:01:23.908937
------------------------------
----> processing file 152 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006230624_00009.zip
==> file: Newsday_20171006230624_00009.zip
==> ID: Newsday_20171006230624_00009
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230624_00009
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230624_00009
==> extract started at 2019-08-07 06:07:25.852099
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006230624_00009.zip
TO uncompressed

==> extract started at 2019-08-07 06:19:59.200822
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006230938_00018.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230938_00018
==> extract completed at 2019-08-07 06:21:19.134644
==> time elapsed: 0:01:19.933822
------------------------------
----> processing file 162 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006230940_00019.zip
==> file: Newsday_20171006230940_00019.zip
==> ID: Newsday_20171006230940_00019
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230940_00019
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230940_00019
==> extract started at 2019-08-07 06:21:19.577718
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006230940_00019.zip
TO uncompressed

==> extract started at 2019-08-07 07:36:09.810857
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231253_00028.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231253_00028
==> extract completed at 2019-08-07 07:37:44.957656
==> time elapsed: 0:01:35.146799
------------------------------
----> processing file 172 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231255_00029.zip
==> file: Newsday_20171006231255_00029.zip
==> ID: Newsday_20171006231255_00029
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231255_00029
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231255_00029
==> extract started at 2019-08-07 07:37:45.435001
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231255_00029.zip
TO uncompressed

==> extract started at 2019-08-07 07:52:35.623414
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231508_00038.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231508_00038
==> extract completed at 2019-08-07 07:54:45.076938
==> time elapsed: 0:02:09.453524
------------------------------
----> processing file 182 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231610_00039.zip
==> file: Newsday_20171006231610_00039.zip
==> ID: Newsday_20171006231610_00039
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231610_00039
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231610_00039
==> extract started at 2019-08-07 07:54:45.769759
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231610_00039.zip
TO uncompressed

==> extract started at 2019-08-07 08:09:11.842103
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231821_00047.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231821_00047
==> extract completed at 2019-08-07 08:10:33.349134
==> time elapsed: 0:01:21.507031
------------------------------
----> processing file 192 of 194
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231923_00048.zip
==> file: Newsday_20171006231923_00048.zip
==> ID: Newsday_20171006231923_00048
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231923_00048
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231923_00048
==> extract started at 2019-08-07 08:10:33.960655
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Newsday/Newsday_20171006231923_00048.zip
TO uncompressed

## Work with uncompressed files

- Back to [Table of Contents](#Table-of-Contents)

Change working directories to the uncompressed paper path.

In [ ]:
%cd $uncompressed_paper_path

In [ ]:
%ls

## load a file into memory.

- Back to [Table of Contents](#Table-of-Contents)

Load one of the files into memory and see what we can do with it.  Beautiful Soup?

Looks like the root element is "Record", then the high-level type of the article is "ObjectType".

ObjectType values:

- Advertisement
- ...

Good options for XML parser:

- `lxml.etree` - [https://stackoverflow.com/questions/12290091/reading-xml-file-and-fetching-its-attributes-value-in-python](https://stackoverflow.com/questions/12290091/reading-xml-file-and-fetching-its-attributes-value-in-python)
- `xmltodict` - [https://docs.python-guide.org/scenarios/xml/](https://docs.python-guide.org/scenarios/xml/)
- `beautifulsoup` using `lxml`

In [ ]:
# loop over files in the current archive folder path.

# declare variables
xml_file_list = None
xml_file_path = None
xml_file = None
xml_dict = None
xml_file_counter = None
object_type_to_count_map = None
object_type_count = None
record_node = None
object_type_node = None
object_type_list = None
object_type = None

# declare variables - auditing
xml_file_counter = None
no_record_counter = None
no_object_type_counter = None
no_object_type_text_counter = None

# init
object_type_to_count_map = {}

# get file list.
xml_file_list = glob.glob( "{}/*.xml".format( uc_archive_folder_path ) )

# loop
xml_file_counter = 0
no_record_counter = 0
no_object_type_counter = 0
no_object_type_text_counter = 0
for xml_file_path in xml_file_list:
    
    xml_file_counter += 1
    
    # try to parse the file
    with open( xml_file_path ) as xml_file:
    
        # parse XML
        xml_dict = xmltodict.parse( xml_file.read() )
        
        # get root.Record.ObjectType value
        record_node = xml_dict.get( "Record", None )
        
        if ( record_node is not None ):
            
            # get object type (looks like xmltodict stores
            #     elements with no attributes and no child
            #     elements just as a string contents mapped
            #     to name in parent, no dictionary)
            #
            # so for:
            # <Record>
            #     ...
            #     <ObjectType>Advertisement</ObjectType>
            #     ...
            # </Record>
            #
            # to get value:
            #     record_node = xml_dict.get( "Record", None )
            #     object_type_list = record_node.get( "ObjectType", None )
            #     object_type = "|".join( object_type_list )
            #
            # NOT:
            #     record_node = xml_dict.get( "Record", None )
            #     object_type_node = record_node.get( "ObjectType", None )
            #     object_type = object_type_node.get( "#text", None )
            #
            # Doc that led me astray: https://docs.python-guide.org/scenarios/xml/
            object_type_list = record_node.get( "ObjectType", None )
            object_type = "|".join( object_type_list )

            # got a type?
            if ( ( object_type is not None ) and ( object_type != "" ) ):

                # we do.  Increment count.
                object_type_count = object_type_to_count_map.get( object_type, 0 )
                object_type_count += 1
                object_type_to_count_map[ object_type ] = object_type_count

            else:

                # object type is None
                no_object_type_text_counter += 1

            #-- END check for type value --#
            
        else:
            
            # increment counter
            no_record_counter += 1
            
        #-- END check if we found a "Record" node in root --#

    #-- END with open( xml_file_path )...: --#
    
#-- END loop over XML files --#

print( "XML file count: {}".format( len( xml_file_list ) ) )
print( "Counters:" )
print( "- Processed {} files".format( xml_file_counter ) )
print( "- No Record: {}".format( no_record_counter ) )
print( "- No ObjectType: {}".format( no_object_type_counter ) )
print( "- No ObjectType value: {}".format( no_object_type_text_counter ) )
print( "\nObjectType values and occurrence counts:")
for object_type, object_type_count in six.iteritems( object_type_to_count_map ):
    
    # print type and count
    print( "- {}: {}".format( object_type, object_type_count ) )
    
#-- END loop over object types. --#

## build list of all ObjectTypes

- Back to [Table of Contents](#Table-of-Contents)

Loop over all folders in the paper path.  For each folder, grab all files in the folder.  For each file, parse XML, then get the ObjectType value and if it isn't already in map of obect types to counts, add it.  Increment count.

In [10]:
xml_folder_list = glob.glob( "{}/*".format( uncompressed_paper_path ) )
print( "folder_list: {}".format( xml_folder_list ) )

folder_list: ['/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220113_00001', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220215_00002', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220218_00003', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220219_00004', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220221_00005', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220324_00006', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220325_00007', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220327_00008', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220331_00009', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220433_00010', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220435_00011', '/mnt/h

In [11]:
# declare variables
xml_file_list = None
xml_file_path = None
xml_file = None
xml_dict = None
xml_file_counter = None
object_type_to_count_map = None
object_type_count = None
record_node = None
object_type_node = None
object_type_list = None
object_type = None

# declare variables - folders
xml_folder_list = None
xml_folder_path = None
xml_folder_count = None
xml_folder_counter = None
xml_folder_start_time = None
xml_folder_end_time = None
xml_folder_duration = None

# declare variables - auditing
xml_file_count = None
xml_file_counter = None
no_record_counter = None
no_object_type_counter = None
no_object_type_text_counter = None

# init
object_type_to_count_map = {}

# first, get the list of folders for the current paper.
xml_folder_list = glob.glob( "{}/*".format( uncompressed_paper_path ) )
xml_folder_count = len( xml_folder_list )

log_message = "Processing {} XML folders in {}".format( xml_folder_count, uncompressed_paper_path )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )

# loop over the folders
xml_folder_counter = 0
for xml_folder_path in xml_folder_list:
    
    xml_folder_counter += 1
    
    # log the folder
    xml_folder_start_time = datetime.datetime.now()
    log_message = "==> Processing XML folder {} ( {} of {} ) @ {}".format( xml_folder_path, xml_folder_counter, xml_folder_count, xml_folder_start_time )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # get file list.
    xml_file_list = glob.glob( "{}/*.xml".format( xml_folder_path ) )
    xml_file_count = len( xml_file_list )

    # log the count
    log_message = "----> XML file count: {}".format( xml_file_count )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # loop
    xml_file_counter = 0
    no_record_counter = 0
    no_object_type_counter = 0
    no_object_type_text_counter = 0
    for xml_file_path in xml_file_list:

        xml_file_counter += 1

        # try to parse the file
        with open( xml_file_path ) as xml_file:

            # parse XML
            xml_dict = xmltodict.parse( xml_file.read() )

            # get root.Record.ObjectType value
            record_node = xml_dict.get( "Record", None )

            if ( record_node is not None ):

                # get object type (looks like xmltodict stores
                #     elements with no attributes and no child
                #     elements just as a string contents mapped
                #     to name in parent, no dictionary)
                #
                # so for:
                # <Record>
                #     ...
                #     <ObjectType>Advertisement</ObjectType>
                #     ...
                # </Record>
                #
                # to get value:
                #     record_node = xml_dict.get( "Record", None )
                #     object_type_list = record_node.get( "ObjectType", None )
                #     object_type = "|".join( object_type_list )
                #
                # NOT:
                #     record_node = xml_dict.get( "Record", None )
                #     object_type_node = record_node.get( "ObjectType", None )
                #     object_type = object_type_node.get( "#text", None )
                #
                # Doc that led me astray: https://docs.python-guide.org/scenarios/xml/
                object_type_list = record_node.get( "ObjectType", None )
                object_type = "|".join( object_type_list )

                # got a type?
                if ( ( object_type is not None ) and ( object_type != "" ) ):

                    # we do.  Increment count.
                    object_type_count = object_type_to_count_map.get( object_type, 0 )
                    object_type_count += 1
                    object_type_to_count_map[ object_type ] = object_type_count

                else:

                    # object type is None
                    no_object_type_text_counter += 1

                #-- END check for type value --#

            else:

                # increment counter
                no_record_counter += 1

            #-- END check if we found a "Record" node in root --#

        #-- END with open( xml_file_path )...: --#

    #-- END loop over XML files --#
    
    # log the folder
    xml_folder_end_time = datetime.datetime.now()
    xml_folder_duration = xml_folder_end_time - xml_folder_start_time
    log_message = "----> Processing complete @ {} ( duration {} )\n".format( xml_folder_end_time, xml_folder_duration )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

#-- END loop over XML directories --#

log_message = "XML file count: {}".format( len( xml_file_list ) )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "Counters:"
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- Processed {} files".format( xml_file_counter )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- No Record: {}".format( no_record_counter )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- No ObjectType: {}".format( no_object_type_counter )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- No ObjectType value: {}".format( no_object_type_text_counter )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "\nObjectType values and occurrence counts:"
my_logging_helper.output_debug_message( log_message, do_print_IN = True )

for object_type, object_type_count in six.iteritems( object_type_to_count_map ):
    
    # print type and count
    log_message = "- {}: {}".format( object_type, object_type_count )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

#-- END loop over object types. --#

Processing 194 XML folders in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday
==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220113_00001 ( 1 of 194 ) @ 2019-08-09 04:03:59.395086
----> XML file count: 25000
----> Processing complete @ 2019-08-09 04:05:42.576368 ( duration 0:01:43.181282 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220215_00002 ( 2 of 194 ) @ 2019-08-09 04:05:42.576758
----> XML file count: 25000
----> Processing complete @ 2019-08-09 04:07:23.952835 ( duration 0:01:41.376077 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220218_00003 ( 3 of 194 ) @ 2019-08-09 04:07:23.953187
----> XML file count: 25000
----> Processing complete @ 2019-08-09 04:09:13.327151 ( duration 0:01:49.373964 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006220219_00004 ( 4

----> XML file count: 25000
----> Processing complete @ 2019-08-09 04:56:18.179286 ( duration 0:01:26.913951 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222319_00018 ( 32 of 194 ) @ 2019-08-09 04:56:18.180042
----> XML file count: 25000
----> Processing complete @ 2019-08-09 04:57:44.328665 ( duration 0:01:26.148623 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222321_00019 ( 33 of 194 ) @ 2019-08-09 04:57:44.329117
----> XML file count: 25000
----> Processing complete @ 2019-08-09 04:59:08.763580 ( duration 0:01:24.434463 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006222323_00020 ( 34 of 194 ) @ 2019-08-09 04:59:08.764287
----> XML file count: 25000
----> Processing complete @ 2019-08-09 05:00:34.775817 ( duration 0:01:26.011530 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/News

----> XML file count: 25000
----> Processing complete @ 2019-08-09 05:39:23.395267 ( duration 0:01:23.287103 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006223513_00006 ( 63 of 194 ) @ 2019-08-09 05:39:23.395629
----> XML file count: 25000
----> Processing complete @ 2019-08-09 05:40:47.452855 ( duration 0:01:24.057226 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006223514_00007 ( 64 of 194 ) @ 2019-08-09 05:40:47.453204
----> XML file count: 25000
----> Processing complete @ 2019-08-09 05:42:11.615027 ( duration 0:01:24.161823 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006223516_00008 ( 65 of 194 ) @ 2019-08-09 05:42:11.615391
----> XML file count: 25000
----> Processing complete @ 2019-08-09 05:43:36.488091 ( duration 0:01:24.872700 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/News

----> XML file count: 25000
----> Processing complete @ 2019-08-09 06:23:29.000239 ( duration 0:01:32.411353 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006224456_00037 ( 94 of 194 ) @ 2019-08-09 06:23:29.001113
----> XML file count: 25000
----> Processing complete @ 2019-08-09 06:24:54.830245 ( duration 0:01:25.829132 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006224501_00040 ( 95 of 194 ) @ 2019-08-09 06:24:54.830756
----> XML file count: 25000
----> Processing complete @ 2019-08-09 06:26:59.717557 ( duration 0:02:04.886801 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006224557_00038 ( 96 of 194 ) @ 2019-08-09 06:26:59.717913
----> XML file count: 25000
----> Processing complete @ 2019-08-09 06:29:17.366163 ( duration 0:02:17.648250 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/News

----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:07:40.408140 ( duration 0:01:23.095156 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006225644_00026 ( 125 of 194 ) @ 2019-08-09 07:07:40.408538
----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:09:02.180956 ( duration 0:01:21.772418 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006225645_00027 ( 126 of 194 ) @ 2019-08-09 07:09:02.181398
----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:10:39.615592 ( duration 0:01:37.434194 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006225646_00028 ( 127 of 194 ) @ 2019-08-09 07:10:39.615959
----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:12:01.512337 ( duration 0:01:21.896378 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/N

----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:51:30.416250 ( duration 0:01:24.284742 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230730_00013 ( 156 of 194 ) @ 2019-08-09 07:51:30.417161
----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:52:54.743202 ( duration 0:01:24.326041 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230732_00014 ( 157 of 194 ) @ 2019-08-09 07:52:54.743554
----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:54:17.987950 ( duration 0:01:23.244396 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006230834_00015 ( 158 of 194 ) @ 2019-08-09 07:54:17.988313
----> XML file count: 25000
----> Processing complete @ 2019-08-09 07:55:41.668603 ( duration 0:01:23.680290 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/N

----> XML file count: 25000
----> Processing complete @ 2019-08-09 08:33:48.897395 ( duration 0:01:24.517842 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231716_00043 ( 187 of 194 ) @ 2019-08-09 08:33:48.897756
----> XML file count: 25000
----> Processing complete @ 2019-08-09 08:35:14.472155 ( duration 0:01:25.574399 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231717_00044 ( 188 of 194 ) @ 2019-08-09 08:35:14.472526
----> XML file count: 25000
----> Processing complete @ 2019-08-09 08:36:39.051114 ( duration 0:01:24.578588 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/Newsday_20171006231818_00045 ( 189 of 194 ) @ 2019-08-09 08:36:39.051971
----> XML file count: 25000
----> Processing complete @ 2019-08-09 08:38:02.147913 ( duration 0:01:23.095942 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/Newsday/N

### Build sorted list of object types

- Back to [Table of Contents](#Table-of-Contents)

In [11]:
# put the raw output from above in a list
raw_object_type_list = [ 'A|d|v|e|r|t|i|s|e|m|e|n|t: 1559817', 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n: 554638', 'Feature|Article: 1358917', 'Classified Advertisement|Advertisement: 392526', 'Editorial|Commentary: 27673', 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h: 120347', 'B|i|r|t|h| |N|o|t|i|c|e: 2327', 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y: 39820', 'I|l|l|u|s|t|r|a|t|i|o|n: 86790', 'Table of Contents|Front Matter: 48620', 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t: 33857', 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c: 15777', 'News|Marriage Announcement: 23473', 'News|Legal Notice: 96674', 'S|t|o|c|k| |Q|u|o|t|e: 15234', 'N|e|w|s: 38501', 'O|b|i|t|u|a|r|y: 50061', 'Letter to the Editor|Correspondence: 41487', 'R|e|v|i|e|w: 69589', 'News|Military/War News: 191', 'Article|Feature: 139126', 'Front Matter|Table of Contents: 9106', 'U|n|d|e|f|i|n|e|d: 15', 'Marriage Announcement|News: 211', 'Correspondence|Letter to the Editor: 4488', 'Commentary|Editorial: 3837', 'Legal Notice|News: 6092', 'Advertisement|Classified Advertisement: 19144', 'Table Of Contents|Front Matter: 2121' ]

In [12]:
# declare variables
#raw_object_type_list = None
raw_object_type = None
object_type_part_list = None
object_type_to_count_map = None
object_type_list = None
object_type_value = None
object_type_count_string = None
object_type_count = None

# loop
object_type_list = []
object_type_to_count_map = {}
for raw_object_type in raw_object_type_list:
    
    # split on colon
    object_type_part_list = raw_object_type.split( ":" )
    
    # object type value - take the first thing, strip off spaces, and add it to list.
    object_type_value = object_type_part_list[ 0 ]
    object_type_value = object_type_value.strip()

    # object type value count - item 2 (index 1)
    object_type_count_string = object_type_part_list[ 1 ]
    object_type_count_string = object_type_count_string.strip()
    object_type_count = int( object_type_count_string )
    
    # add value to list.
    if ( object_type_value not in object_type_list ):
        
        # add it.
        object_type_list.append( object_type_value )
        
    else:
        
        # error.
        print( "ERROR - object type value {} in list more than once.  Hmmm.".format( object_type_value ) )
        
    #-- END check to see if value already in list. --#
    
    # add count to map.
    if ( object_type_value not in object_type_to_count_map ):
        
        # add count.
        object_type_to_count_map[ object_type_value ] = object_type_count
        
    else:
        
        # error.
        print( "ERROR - object type value {} already has count in map.  Hmmm.".format( object_type_value ) )
        
    #-- END check to see if value already in list. --#
    
#-- END loop over raw object types --#

# sort the list of object types
object_type_list.sort()

print( object_type_list )

['Advertisement|Classified Advertisement', 'Article|Feature', 'A|d|v|e|r|t|i|s|e|m|e|n|t', 'B|i|r|t|h| |N|o|t|i|c|e', 'Classified Advertisement|Advertisement', 'Commentary|Editorial', 'Correspondence|Letter to the Editor', 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t', 'Editorial|Commentary', 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c', 'Feature|Article', 'Front Matter|Table of Contents', 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y', 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n', 'I|l|l|u|s|t|r|a|t|i|o|n', 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h', 'Legal Notice|News', 'Letter to the Editor|Correspondence', 'Marriage Announcement|News', 'News|Legal Notice', 'News|Marriage Announcement', 'News|Military/War News', 'N|e|w|s', 'O|b|i|t|u|a|r|y', 'R|e|v|i|e|w', 'S|t|o|c|k| |Q|u|o|t|e', 'Table Of Contents|Front Matter', 'Table of Contents|Front Matter', 'U|n|d|e|f|i|n|e|d']


    ['Advertisement|Classified Advertisement', 'Article|Feature', 'A|d|v|e|r|t|i|s|e|m|e|n|t', 'B|i|r|t|h| |N|o|t|i|c|e', 'Classified Advertisement|Advertisement', 'Commentary|Editorial', 'Correspondence|Letter to the Editor', 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t', 'Editorial|Commentary', 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c', 'Feature|Article', 'Front Matter|Table of Contents', 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y', 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n', 'I|l|l|u|s|t|r|a|t|i|o|n', 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h', 'Legal Notice|News', 'Letter to the Editor|Correspondence', 'Marriage Announcement|News', 'News|Legal Notice', 'News|Marriage Announcement', 'News|Military/War News', 'N|e|w|s', 'O|b|i|t|u|a|r|y', 'R|e|v|i|e|w', 'S|t|o|c|k| |Q|u|o|t|e', 'Table Of Contents|Front Matter', 'Table of Contents|Front Matter', 'U|n|d|e|f|i|n|e|d']

# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- create bash script to uncompress all files in a folder from data to uncompressed, when passed paper and archive identifiers?